In [11]:
import json
from dotenv import load_dotenv
import os
from pydantic import ValidationError
from openai import OpenAI
from datetime import datetime
from fastapi.encoders import jsonable_encoder
from TransactionResponse import Response


EXECUTION_DATETIME = datetime.now().strftime('%Y%m%d_%H%M%S') # será compartido en toda la ejecución para referenciar el mismo archivo en todas las capas

today_date = datetime.today().strftime('%Y-%m-%d')

user_input_lang = 'spanish'

# aqui tiene que ir el correspondiente archivo de la carpeta landing
user_input_text = """
acabamos de comprar en costco un par de jeans por 500 unos cuernitos por 99 dos pastelitos por 150 cada uno 25 litros de gasolina por 23.01 cada uno 1.30 kilos de tomate en 29.90 cada uno pagado con tarjeta de crédito costco citibanamex un total de 1500.12 pesos y con tarjeta NU un total de 2000.01 pesos
"""

load_dotenv(dotenv_path='./.env')

# Transform response in py dict
# response_json = jsonable_encoder(response)['transaction']
# print(response_json)

False

In [ ]:
client = OpenAI(
    base_url="https://api.moonshot.ai/v1",
    api_key=os.getenv('MOONSHOT_API_KEY'),
)

In [ ]:
chat_completion = client.chat.completions.create(
        model="kimi-k2.5",
        # messages=[{"role": "user", "content": "Hello"}],
        messages=[
        {"role": "system", "content": f"""
You are an expert at structured data extraction. You will be given unstructured text with detailed receipts from purchases in commerces and detailed payment methods of that specific receipt and should convert it into the given structure. Consider todays date as {today_date} and consider you will receive the data in {user_input_lang} but the structure is in english. Return your analysis in valid JSON matching this schema: {Response.model_json_schema()}
        """ },
        {"role": "user", "content": user_input_text}
    ],
        response_format={"type": "json_object"}
)


In [4]:
print(chat_completion)

NameError: name 'chat_completion' is not defined

In [ ]:
# Manually parse the output into Pydantic
try:
    raw_content = chat_completion.choices[0].message.content
    transaction = Response.model_validate_json(raw_content)
except ValidationError as e:
    print(f"LLM output failed validation: {e}")

In [25]:
raw_content = chat_completion.choices[0].message.content
# response_json = jsonable_encoder(raw_content)['transaction']
print(raw_content)

{
  "transaction": {
    "transaction_date": "2026-02-05",
    "commerce": "Costco",
    "receipt_detail": [
      {
        "quantity": 1,
        "description": "Jeans",
        "category": "Clothing",
        "unit_price": 500,
        "subtotal": 500
      },
      {
        "quantity": 1,
        "description": "Cuernitos",
        "category": "Bakery",
        "unit_price": 99,
        "subtotal": 99
      },
      {
        "quantity": 2,
        "description": "Pastelitos",
        "category": "Bakery",
        "unit_price": 150,
        "subtotal": 300
      },
      {
        "quantity": 25,
        "description": "Gasolina",
        "category": "Fuel",
        "unit_price": 23.01,
        "subtotal": 575.25
      },
      {
        "quantity": 1.30,
        "description": "Tomate",
        "category": "Groceries",
        "unit_price": 29.90,
        "subtotal": 38.87
      }
    ],
    "receipt_payment_method": [
      {
        "payment_method_type": "Credit Card",
       

In [27]:
# Save into to_text layer
today_date_folder = datetime.today().strftime('%Y%m%d')
os.makedirs(f'./spend/text_to_json/{today_date_folder}',exist_ok=True)

In [28]:
filename = f'receipt_payments_{EXECUTION_DATETIME}.json'
filepath_json = f'./spend/text_to_json/{today_date_folder}/{filename}'

with open(filepath_json, 'w') as f:
    json.dump(raw_content, f)

In [ ]:
%store filename
%store filepath_json
%store today_date_folder
%store EXECUTION_DATETIME

UsageError: Unknown variable 'filename'
